## Code Modules

In [1]:
import numpy as np,pandas as pd
import h5py,pylab as pl
import tensorflow_hub as th,tensorflow as tf
from tensorflow import image as timage

## Data

In [2]:
fpath='../input/tf-cats-vs-dogs/'
f='CatDogImages.h5'
f=h5py.File(fpath+f,'r')
keys=list(f.keys()); print(keys)
x_test=np.array(f[keys[0]])
y_test=np.array(f[keys[1]],dtype='int8')
x_train=np.array(f[keys[2]])
y_train=np.array(f[keys[3]],dtype='int8')
N=len(y_train); shuffle_ids=np.arange(N)
np.random.RandomState(12).shuffle(shuffle_ids)
x_train,y_train=x_train[shuffle_ids],y_train[shuffle_ids]
N=len(y_test); shuffle_ids=np.arange(N)
np.random.RandomState(23).shuffle(shuffle_ids)
x_test,y_test=x_test[shuffle_ids],y_test[shuffle_ids]
n=int(len(x_test)/2)
x_valid,y_valid=x_test[:n],y_test[:n]
x_test,y_test=x_test[n:],y_test[n:]
pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
              [x_train.dtype,x_valid.dtype,x_test.dtype],
              [y_train.shape,y_valid.shape,y_test.shape],
              [y_train.dtype,y_valid.dtype,y_test.dtype]],
             columns=['train','valid','test'],
             index=['image shape','image type',
                    'label shape','label type'])

['test_images', 'test_labels', 'train_images', 'train_labels']


,train,valid,test
image shape,"(18610, 128, 128, 3)","(2326, 128, 128, 3)","(2326, 128, 128, 3)"
image type,float32,float32,float32
label shape,"(18610, 1)","(2326, 1)","(2326, 1)"
label type,int8,int8,int8


## TF Hub Models

In [3]:
fw='weights.best.hdf5'
def premodel(pix,den,mh,lbl,activ,loss):
    model=tf.keras.Sequential([
        tf.keras.layers.Input((pix,pix,3),
                              name='input'),
        th.KerasLayer(mh,trainable=True),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(den,activation='relu'),
        tf.keras.layers.Dropout(rate=.5),
        tf.keras.layers.Dense(lbl,activation=activ)])
    model.compile(optimizer='adam',
                  metrics=['accuracy'],loss=loss)
    display(model.summary())
    return model
def cb(fw):
    early_stopping=tf.keras.callbacks\
    .EarlyStopping(monitor='val_loss',patience=20,verbose=2)
    checkpointer=tf.keras.callbacks\
    .ModelCheckpoint(filepath=fw,save_best_only=True,verbose=2)
    lr_reduction=tf.keras.callbacks\
    .ReduceLROnPlateau(monitor='val_loss',verbose=2,
                       patience=5,factor=.8)
    return [checkpointer,early_stopping,lr_reduction]

In [4]:
[handle_base,pixels]=["mobilenet_v1_100_128",128]
mhandle="https://tfhub.dev/google/imagenet/{}/feature_vector/4"\
.format(handle_base)

In [5]:
model=premodel(pixels,3072,mhandle,1,
               'sigmoid','binary_crossentropy')
history=model.fit(x=x_train,y=y_train,batch_size=128,
                  epochs=3,callbacks=cb(fw),
                  validation_data=(x_valid,y_valid))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1024)              3228864   
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 3072)              3148800   
_________________________________________________________________
dropout (Dropout)            (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3073      
Total params: 6,380,737
Trainable params: 6,358,849
Non-trainable params: 21,888
_________________________________________________________________


None

Epoch 1/3
146/146 [==============================] - ETA: 0s - loss: 0.2822 - accuracy: 0.9486
Epoch 00001: val_loss improved from inf to 0.26081, saving model to weights.best.hdf5
146/146 [==============================] - 27s 183ms/step - loss: 0.2822 - accuracy: 0.9486 - val_loss: 0.2608 - val_accuracy: 0.9686 - lr: 0.0010
Epoch 2/3
146/146 [==============================] - ETA: 0s - loss: 0.1831 - accuracy: 0.9804
Epoch 00002: val_loss did not improve from 0.26081
146/146 [==============================] - 25s 174ms/step - loss: 0.1831 - accuracy: 0.9804 - val_loss: 0.5743 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 3/3
146/146 [==============================] - ETA: 0s - loss: 0.1629 - accuracy: 0.9872
Epoch 00003: val_loss did not improve from 0.26081
146/146 [==============================] - 26s 176ms/step - loss: 0.1629 - accuracy: 0.9872 - val_loss: 0.3174 - val_accuracy: 0.9600 - lr: 0.0010


In [6]:
model.load_weights(fw)
model.evaluate(x_test,y_test)

73/73 [==============================] - 1s 11ms/step - loss: 0.2531 - accuracy: 0.9635


[0.2531206011772156, 0.9634565711021423]